**1. Self Attention📌**

아래 코드를 수행해보고, Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명: self attention은 입력 시퀀스의 각 요소가 다른 모든 요소와 상호작용하여 중요한 정보를 학습할 수 있게 하는 메커니즘이다. 이는 각 단어가 문맥을 이해하고, 해당 문맥 내에서 자신이 얼마나 중요한지를 결정할 수 있게 한다. 먼저, 입력 벡터에서 Query, Key, Value를 생성한다. 이후 Query와 Key 사이의 연관성을 계산하기 위해 두 값을 내적하여 Attention score을 계산한다. 그리고 값들을 정규화시키기 위해 softmax 함수를 거쳐 Attention Weight를 계산한 후, Attention Weight와 Value의 가중합을 통해 최종 출력을 생성한다.

In [1]:
import numpy as np

# 간단한 문장: ['나는', '사과를', '먹었다']
words = ['나는', '사과를', '먹었다']
word_vectors = {
    '나는': np.array([1, 0, 0]),
    '사과를': np.array([0, 1, 0]),
    '먹었다': np.array([0, 0, 1])
}

# 유사도 계산 함수 (self-attention)
def self_attention(query, key): # Query와 Key를 내적하여 Attention score 계산
    return np.dot(query, key)

attention_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        attention_matrix[i][j] = self_attention(word_vectors[words[i]], word_vectors[words[j]]) # 단어 i와 단어 j의 벡터 사이의 self-attention 점수를 계산하여 행렬에 저장

print("Self-Attention Matrix:") # 결과 출력
print(attention_matrix)


Self-Attention Matrix:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


**2. Multi-Head Self Attention📌**

아래 코드를 수행해보고, Multi-Head Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명: Multi-Head Self Attention은 입력 시퀀스의 각 요소가 여러 가지 시각에서 상호작용하여 더 다양한 정보를 학습할 수 있게 하는 메커니즘이다. 기본적인 self-attention과 동일하게 Query, Key, Value를 사용하지만, 이를 여러 개의 head로 나누어 독립적으로 attention을 수행한다. 각 head는 서로 다른 정보에 집중하며, 각기 다른 Query, Key, Value 벡터를 생성해 각각의 Attention Score를 계산한다. 이렇게 여러 head에서 나온 Attention Score를 기반으로 한 다양한 Attention Weight가 생성되며, 이 Weight와 Value를 사용해 각각의 출력을 생성한 후, 이 출력들을 결합하여 최종 결과를 만든다. 이는 모델이 다양한 문맥 정보를 학습할 수 있게 하고, 더 풍부한 표현을 만들 수 있게 해준다.

In [2]:
# 여러 개의 attention heads
def multi_head_self_attention(query, key, heads=3): # 여러 개의 heads에 대해 Query와 Key를 내적하여 Attention Score 계산
    return [np.dot(query, key) for _ in range(heads)]

multi_head_attention_matrix = np.zeros((len(words), len(words), 3))
for i in range(len(words)):
    for j in range(len(words)):
        multi_head_attention_matrix[i][j] = multi_head_self_attention(word_vectors[words[i]], word_vectors[words[j]])  # 각 단어 쌍에 대해 다수의 attention head 결과 계산

print("\nMulti-Head Self-Attention Matrix:") # 결과 출력
print(multi_head_attention_matrix)



Multi-Head Self-Attention Matrix:
[[[1. 1. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [1. 1. 1.]]]


**3. Masked Multi-Head Self Attention📌**

아래 코드를 수행해보고, Masked Multi-Head Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명: Masked Multi-Head Self Attention은 미래의 정보가 현재의 계산에 영향을 미치지 않도록 입력 시퀀스에서 특정 위치의 정보를 가리는 메커니즘이다. 주로 시퀀스 생성 모델에서 사용되며, 모델이 순차적으로 다음 토큰을 예측할 때 미래의 단어를 미리 알 수 없도록 하기 위해 사용된다. 기본적인 Multi-Head Self Attention처럼 여러 개의 Query, Key, Value 벡터를 생성하고 여러 head에서 독립적인 attention을 수행하지만, mask를 적용하여 미래의 단어에 대한 연산을 무효화한다. 이를 위해 Query와 Key 간의 Attention Score 계산 시, 미래 토큰에 해당하는 위치에 큰 음수를 더해 softmax를 적용한 후, 그 값이 0에 가깝게 만들어 해당 위치의 영향을 차단한다. 이후 각 head에서 계산된 Attention Weight와 Value의 가중합을 통해 출력을 생성하고, 여러 head의 출력을 결합하여 최종 결과를 만든다. 이 방식은 모델이 시퀀스를 순차적으로 생성할 때 미래 정보 없이 이전 단어들만을 고려하도록 보장한다.

In [3]:
# 마스크 추가: 현재 단어 이후의 단어는 계산하지 않음
def masked_attention(query, key, mask): # Query와 Key의 내적을 계산, 마스크 값에 따라 결과를 제한
    return np.dot(query, key) * mask

mask = np.array([1, 1, 0])  # 첫 번째, 두 번째는 보지만, 세 번째는 보지 않음
masked_attention_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        masked_attention_matrix[i][j] = masked_attention(word_vectors[words[i]], word_vectors[words[j]], mask[j]) # i번째 단어를 query로, j번째 단어를 key로 사용하며, 마스크를 적용해 attention score 계산

print("\nMasked Self-Attention Matrix:") # 결과 출력
print(masked_attention_matrix)



Masked Self-Attention Matrix:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]


**4. Cross Attention📌**

아래 코드를 수행해보고, Cross Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명: Cross Attention은 두 가지 서로 다른 입력 시퀀스 간의 상호작용을 학습하는 메커니즘이다. 이 메커니즘은 한 시퀀스에서 Query를 생성하고, 다른 시퀀스에서 Key와 Value를 생성하여 두 시퀀스 간의 연관성을 파악한다. 즉, 자기 자신이 아닌 다른 시퀀스의 정보에 집중하게 된다. Query와 Key 간의 Attention Score를 계산하여 두 시퀀스 사이의 상호작용을 측정하고, 이 Score를 softmax로 정규화해 Attention Weight를 만든다. 이 Weight는 Key 시퀀스와 연관된 정보의 중요도를 나타내며, 이를 사용해 Value 시퀀스의 정보를 가중합해 최종 출력을 생성한다. Cross Attention은 다양한 시퀀스 간의 상호 관계를 학습하는 데 유용하며, 대표적으로 Transformer 기반 모델에서 인코더-디코더 구조에서 사용된다. 인코더의 출력이 디코더의 Query와 상호작용하며, 디코더는 주어진 시퀀스와 관련된 중요한 정보를 인코더의 출력에서 추출하게 된다.








In [4]:
# 입력 문장과 응답 문장
question_words = ['너는', '사과를']
answer_words = ['나는', '먹었다']
question_vectors = {
    '너는': np.array([1, 0]),
    '사과를': np.array([0, 1])
}
answer_vectors = {
    '나는': np.array([1, 0]),
    '먹었다': np.array([0, 1])
}

# Cross-Attention
cross_attention_matrix = np.zeros((len(question_words), len(answer_words)))
for i in range(len(question_words)):
    for j in range(len(answer_words)):
        cross_attention_matrix[i][j] = np.dot(question_vectors[question_words[i]], answer_vectors[answer_words[j]])  # i번째 질문 단어와 j번째 응답 단어의 벡터 간의 내적을 계산하여 attention score 구함

print("\nCross-Attention Matrix:") # 결과 출력
print(cross_attention_matrix)


Cross-Attention Matrix:
[[1. 0.]
 [0. 1.]]
